# Assignment 2
## Changhong Li
## 23333239

In [1]:
! pip install qiskit
! pip install qiskit-aer
! pip install matplotlib
! pip install numpy
! pip install qiskit-ibm-runtime
! pip install jupyter

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.7 MB/s  0:00:00
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 11.6 MB/s  0:00:004.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18/18 [qiskit-ibm-runtime]m 17/18 [qiskit-ibm-runtime]
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jupyter_lsp-2.3.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached jup

## 2.1 量子隐形传态（Quantum Teleportation）

In [3]:
# qiskit_algorithms_demo.py
# 实现：1) 量子隐形传态  2) 超密编码  3) 德意志算法
# 兼容 Qiskit 1.x（Aer 需单独安装：qiskit-aer）
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import math
from collections import Counter


# -----------------------------
# 公用：运行电路并返回测量计数
# -----------------------------
def run_circuit(qc: QuantumCircuit, shots: int = 1024):
    sim = AerSimulator()
    compiled = transpile(qc, sim)
    result = sim.run(compiled, shots=shots).result()
    return result.get_counts()


# -----------------------------
# 1) 量子隐形传态
# -----------------------------
def prepare_arbitrary_state(qc: QuantumCircuit, q: int, theta: float, phi: float):
    """
    准备一般单比特态：cos(theta/2)|0> + e^{i phi} sin(theta/2)|1>
    通过 Ry(theta) -> Rz(phi) 实现
    """
    qc.ry(theta, q)
    qc.rz(phi, q)


def teleportation_circuit(theta: float, phi: float) -> QuantumCircuit:
    """
    3 个量子比特：q0(待传态)|Alice, q1(Alice), q2(Bob)
    经典位：c0, c1 为 Alice 测量结果；verify 用于在 Bob 侧验证
    验证方法：在 Bob 的 q2 上施加准备门的逆（Rz(-phi), Ry(-theta)），
    若传态正确，q2 测量应几乎总为 |0>
    """
    qr = QuantumRegister(3, "q")
    cr0 = ClassicalRegister(1, "c0")      # 测 q0
    cr1 = ClassicalRegister(1, "c1")      # 测 q1
    cverify = ClassicalRegister(1, "verify")  # 验证 q2
    qc = QuantumCircuit(qr, cr0, cr1, cverify, name="teleportation")

    # 1) 准备未知量子态 |psi> 在 q0
    prepare_arbitrary_state(qc, 0, theta, phi)
    qc.barrier()

    # 2) 共享 EPR 对 (q1, q2)
    qc.h(1)
    qc.cx(1, 2)
    qc.barrier()

    # 3) Alice 的贝尔测量 (q0, q1)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, cr0)
    qc.measure(1, cr1)

    # 4) 基于经典结果的纠正 —— 用 if_test 取代 .c_if
    with qc.if_test((cr1, 1)):
        qc.x(2)
    with qc.if_test((cr0, 1)):
        qc.z(2)
    qc.barrier()


    # 5) 验证：施加准备门的逆到 q2，再测量
    qc.rz(-phi, 2)
    qc.ry(-theta, 2)
    qc.measure(2, cverify)

    return qc


def demo_teleportation():
    # 选一组非平凡参数
    theta = 1.0  # 弧度
    phi = 0.7
    qc = teleportation_circuit(theta, phi)
    counts = run_circuit(qc, shots=2048)

    # 计数字串顺序为按寄存器拼接（高位在左）：verify c1 c0
    # 统计 verify 位为 '0' 的概率（成功率应 ~1.0）
    shots = sum(counts.values())
    success = sum(v for k, v in counts.items() if k[0] == "0") / shots
    print("=== Quantum Teleportation ===")
    print("Counts (verify c1 c0):", counts)
    print(f"Teleportation success (verify=0): {success:.4f}\n")


# -----------------------------
# 2) 超密编码 (Superdense Coding)
# -----------------------------
def superdense_coding(bits: str) -> QuantumCircuit:
    """
    Alice 与 Bob 共享贝尔态。Alice 用 1 个量子比特编码 2 位经典比特。
    bits ∈ {'00', '01', '10', '11'}
    """
    if bits not in {"00", "01", "10", "11"}:
        raise ValueError("bits must be one of {'00','01','10','11'}")

    qc = QuantumCircuit(2, 2, name=f"SDC_{bits}")
    # 共享贝尔态（q0: Alice, q1: Bob）
    qc.h(0)
    qc.cx(0, 1)

    # Alice 编码：00->I, 01->X, 10->Z, 11->XZ
    if bits == "01":
        qc.x(0)
    elif bits == "10":
        qc.z(0)
    elif bits == "11":
        qc.x(0)
        qc.z(0)

    # 发送 q0 给 Bob（在电路中等价于 Bob 拿到 q0）
    # Bob 解码
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0, 1], [0, 1])  # 测得的两位应还原为 bits
    return qc


def demo_superdense():
    print("=== Superdense Coding ===")
    for bits in ["00", "01", "10", "11"]:
        qc = superdense_coding(bits)
        counts = run_circuit(qc, shots=1024)
        # 最高频结果应等于原 bits（注意 bit 串顺序为 c1 c0）
        top = max(counts.items(), key=lambda kv: kv[1])[0]
        print(f"Message {bits} -> measured counts (c1 c0): {counts}, top={top}")
    print()


# -----------------------------
# 3) 德意志算法 (Deutsch)
# -----------------------------
def deutsch_oracle(qc: QuantumCircuit, f_type: str):
    """
    在给定电路上实现 U_f:
    |x,y> -> |x, y ⊕ f(x)|
    f_type:
      - 'const0': f(x)=0
      - 'const1': f(x)=1
      - 'balanced_id': f(x)=x
      - 'balanced_not': f(x)=¬x (= x⊕1)
    """
    if f_type == "const0":
        # 恒 0：什么都不做
        pass
    elif f_type == "const1":
        # 恒 1：翻转目标位 y
        qc.x(1)
    elif f_type == "balanced_id":
        # 平衡：f(x)=x -> CNOT 控制 x 到 y
        qc.cx(0, 1)
    elif f_type == "balanced_not":
        # 平衡：f(x)=¬x -> 先对 y 取反，再 CNOT
        qc.x(1)
        qc.cx(0, 1)
    else:
        raise ValueError("Unknown f_type")


def deutsch_circuit(f_type: str) -> QuantumCircuit:
    """
    初态 |0>|1|，H⊗H，U_f，最后对第 1 个量子比特做 H 并测量：
    测得 0 -> 常量函数；测得 1 -> 平衡函数
    """
    qc = QuantumCircuit(2, 1, name=f"Deutsch_{f_type}")
    # |0>|1>
    qc.x(1)
    # H⊗H
    qc.h(0)
    qc.h(1)
    # U_f
    deutsch_oracle(qc, f_type)
    # H ⊗ I，再测量第一个比特
    qc.h(0)
    qc.measure(0, 0)
    return qc


def demo_deutsch():
    print("=== Deutsch Algorithm ===")
    tests = [
        ("const0", "constant"),
        ("const1", "constant"),
        ("balanced_id", "balanced"),
        ("balanced_not", "balanced"),
    ]
    for f_type, label in tests:
        qc = deutsch_circuit(f_type)
        counts = run_circuit(qc, shots=1024)
        # 这里只有 1 位输出（寄存器顺序：c0），'0' 表示常量，'1' 表示平衡
        top = max(counts.items(), key=lambda kv: kv[1])[0]
        print(f"f_type={f_type:12s} -> counts: {counts}, decision={ 'constant' if top=='0' else 'balanced' } (expected {label})")
    print()


# -----------------------------
# 主函数演示
# -----------------------------
if __name__ == "__main__":
    demo_teleportation()
    demo_superdense()
    demo_deutsch()


=== Quantum Teleportation ===
Counts (verify c1 c0): {'0 0 1': 536, '0 1 1': 517, '0 1 0': 485, '0 0 0': 510}
Teleportation success (verify=0): 1.0000

=== Superdense Coding ===
Message 00 -> measured counts (c1 c0): {'00': 1024}, top=00
Message 01 -> measured counts (c1 c0): {'10': 1024}, top=10
Message 10 -> measured counts (c1 c0): {'01': 1024}, top=01
Message 11 -> measured counts (c1 c0): {'11': 1024}, top=11

=== Deutsch Algorithm ===
f_type=const0       -> counts: {'0': 1024}, decision=constant (expected constant)
f_type=const1       -> counts: {'0': 1024}, decision=constant (expected constant)
f_type=balanced_id  -> counts: {'1': 1024}, decision=balanced (expected balanced)
f_type=balanced_not -> counts: {'1': 1024}, decision=balanced (expected balanced)



In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import math
from collections import Counter


# -----------------------------
# 公用：运行电路并返回测量计数
# -----------------------------
def run_circuit(qc: QuantumCircuit, shots: int = 1024):
    sim = AerSimulator()
    compiled = transpile(qc, sim)
    result = sim.run(compiled, shots=shots).result()
    return result.get_counts()


# -----------------------------
# 1) 量子隐形传态
# -----------------------------
def prepare_arbitrary_state(qc: QuantumCircuit, q: int, theta: float, phi: float):
    """
    准备一般单比特态：cos(theta/2)|0> + e^{i phi} sin(theta/2)|1>
    通过 Ry(theta) -> Rz(phi) 实现
    """
    qc.ry(theta, q)
    qc.rz(phi, q)


def teleportation_circuit(theta: float, phi: float) -> QuantumCircuit:
    """
    3 个量子比特：q0(待传态)|Alice, q1(Alice), q2(Bob)
    经典位：c0, c1 为 Alice 测量结果；verify 用于在 Bob 侧验证
    验证方法：在 Bob 的 q2 上施加准备门的逆（Rz(-phi), Ry(-theta)），
    若传态正确，q2 测量应几乎总为 |0>
    """
    qr = QuantumRegister(3, "q")
    cr0 = ClassicalRegister(1, "c0")      # 测 q0
    cr1 = ClassicalRegister(1, "c1")      # 测 q1
    cverify = ClassicalRegister(1, "verify")  # 验证 q2
    qc = QuantumCircuit(qr, cr0, cr1, cverify, name="teleportation")

    # 1) 准备未知量子态 |psi> 在 q0
    prepare_arbitrary_state(qc, 0, theta, phi)
    qc.barrier()

    # 2) 共享 EPR 对 (q1, q2)
    qc.h(1)
    qc.cx(1, 2)
    qc.barrier()

    # 3) Alice 的贝尔测量 (q0, q1)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, cr0)
    qc.measure(1, cr1)

    # 4) 基于经典结果的纠正 —— 用 if_test 取代 .c_if
    with qc.if_test((cr1, 1)):
        qc.x(2)
    with qc.if_test((cr0, 1)):
        qc.z(2)
    qc.barrier()


    # 5) 验证：施加准备门的逆到 q2，再测量
    qc.rz(-phi, 2)
    qc.ry(-theta, 2)
    qc.measure(2, cverify)

    return qc


def demo_teleportation():
    # 选一组非平凡参数
    theta = 1.0  # 弧度
    phi = 0.7
    qc = teleportation_circuit(theta, phi)
    counts = run_circuit(qc, shots=2048)

    # 计数字串顺序为按寄存器拼接（高位在左）：verify c1 c0
    # 统计 verify 位为 '0' 的概率（成功率应 ~1.0）
    shots = sum(counts.values())
    success = sum(v for k, v in counts.items() if k[0] == "0") / shots
    print("=== Quantum Teleportation ===")
    print("Counts (verify c1 c0):", counts)
    print(f"Teleportation success (verify=0): {success:.4f}\n")

demo_teleportation()

=== Quantum Teleportation ===
Counts (verify c1 c0): {'0 1 1': 504, '0 1 0': 500, '0 0 0': 522, '0 0 1': 522}
Teleportation success (verify=0): 1.0000



## 2.2 超密编码（Superdense Coding）

## 2.3 Deutsch 算法（Deutsch Algorithm）